In [1]:
# Imports from the Python standard library
from io import StringIO
from typing import Iterable
import numpy as np
import matplotlib.pyplot as plt
import openff

# Imports from the comp chem ecosystem
from openff.units import Quantity, unit
from openmm import unit as openmm_unit
#from pdbfixer import PDBFixer

# New topology imports
from openff.interchange import Interchange

# Imports from the toolkit
from rdkit import Chem
import MDAnalysis as mda
from MDAnalysis.analysis import distances
from openff.toolkit import ForceField, Molecule, Topology
from openff.toolkit.utils.toolkits import OpenEyeToolkitWrapper
from openff.toolkit.utils import get_data_file_path

# Imports from OpenFF
import openff.nagl
from openff.nagl import GNNModel 
from openff.nagl_models import list_available_nagl_models
# If ypu want a diff toolkit wrapper check out https://docs.openforcefield.org/projects/nagl/en/latest/toolkit_wrappers.html

import pandas as pd
import nglview
import time
import mdtraj
import yaml
import os

### Build your molecules from SMILES strings

In [2]:
list_available_nagl_models()
model_path = '/home/julianne/miniconda3/envs/openff/lib/python3.11/site-packages/openff/nagl_models/models/am1bcc/openff-gnn-am1bcc-0.1.0-rc.2.pt'
model = GNNModel.load(model_path)

In [3]:
hexane = Molecule.from_smiles("C" * 6, allow_undefined_stereo=True)
heptane = Molecule.from_smiles("C" * 7, allow_undefined_stereo=True)
octane = Molecule.from_smiles("C" * 8, allow_undefined_stereo=True)
decane = Molecule.from_smiles("C" * 10, allow_undefined_stereo=True)
pentadecane = Molecule.from_smiles("C" * 15, allow_undefined_stereo=True)
TIP3P = Molecule.from_smiles("O")

In [4]:
alkanes = {'hexane':hexane, 'heptane':heptane, 'octane':octane,'decane':decane, 'pentadecane':pentadecane}
for name, alkane in alkanes.items():

    alkane.generate_conformers()


    nagl_charge = model.compute_property(alkane, check_domains = True, error_if_unsupported=True)
    alkane.partial_charges  = nagl_charge * unit.elementary_charge #openFF units attached to partial charges property 

        
    alkane.name = "ALK"
    for i, atom in enumerate(alkane.atoms, 3):
        atom.metadata["residue_name"] = 'ALK'
    alkane.generate_unique_atom_names() #option to use (suffix = 'x')


    topology = Topology.from_molecules(
        [alkane]
    )

    forcefield = ForceField("openff-2.1.0.offxml")

    interchange = Interchange.from_smirnoff(
        force_field=forcefield,
        topology=topology,
        charge_from_molecules = [alkane]
    )
    interchange

    interchange.to_top(f"{name}.top")
    interchange.to_gro(f"{name}.gro")

/home/julianne/miniconda3/envs/openff_clone/lib/python3.11/site-packages/openff/interchange/interop/gromacs/export/_export.py:48: UserWarning: WARNING: System defined with no box vectors, which GROMACS does not offically support in versions 2020 or newer (see https://gitlab.com/gromacs/gromacs/-/issues/3526). Setting box vectors to a 5  nm cube.
  self._write_gro(gro, decimal)


## Estimate dimensions of box for packmol 

In [4]:
model = '/media/julianne/DATA/Lipids/OpenFFLipid/AlkaneDiffusion/pentadecane.pdb'
view = nglview.show_structure_file(model)
view


NGLWidget()

In [3]:
u = mda.Universe('pentadecane.gro')

head_group = u.select_atoms('name C1x') #select atom in head group, in this case Nitrogen
tail_group = u.select_atoms('name C15x') #select atom in tail group, in this case last carbon in unsaturated tail

distance = distances.distance_array(head_group.positions,tail_group.positions)
print("Length of lipid from head group to tail group is:\n","~",round(distance[0][0],2),'Angstrom') 

Length of lipid from head group to tail group is:
 ~ 17.85 Angstrom


Run packAlkanes.py script to pack them in packmol